In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "23770286"

tripadvisor = Tripadvisor()
# hotels = tripadvisor.get_hotels()
hotel = tripadvisor.hotel_detail(hotel_id)

In [ ]:
import json

print(json.dumps(hotel, indent=4))

In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "20226478"

tripadvisor = Tripadvisor()
review = tripadvisor.hotel_review(hotel_id, 0)

In [ ]:
import json

# print(type(review['paging']['total_results']))
# print(len(review['data']))
print(json.dumps(review['data'], indent=7))
# print(len(review['data']))

In [ ]:
import json

beneran_review = [data for data in review['data'] if 'ad_context' not in data]
len(beneran_review)

In [ ]:
review2 = tripadvisor.hotel_review(hotel_id, 20)

In [ ]:
print(json.dumps(review2['data'][0], indent=7))

# COLLECT HOTEL DETAIL (COMPLETED)

In [ ]:
from tripadvisor import Tripadvisor
import mysql.connector
import time
import config
import os
import json

hotel_data_dir = './hotel_data'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

# select all hotel list that have not been parsed
hotel_list = db_select(db, "SELECT * FROM hotel_list WHERE hotel_detail_parsed=0 LIMIT 254")

# instantiate TripAdvisor object
tripadvisor = Tripadvisor()

# iterate per hotel to get hotel detail then change hotel_detail_parsed to True
for hotel in hotel_list:
    hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed = hotel

    # create hotel dir if not exist
    hotel_id_dir = os.path.join(hotel_data_dir, hotel_id)
    if not os.path.exists(hotel_id_dir):
        os.makedirs(hotel_id_dir)

    # get hotel detail from tripadvisor api
    hotel_detail = tripadvisor.hotel_detail(hotel_id)

    # store hotel_detail data to database and local file
    if 'errors' in hotel_detail.keys():
            print(f'can\'t parse hotel with id {hotel["hotel_id"]}')
    else:
        #store and update database
        cursor.execute(
            'INSERT IGNORE INTO hotel_detail (hotel_id, hotel_detail) VALUES (%s, %s)',
            (hotel_id, json.dumps(hotel_detail))
        )
        cursor.execute(
            f'UPDATE hotel_list SET hotel_detail_parsed=1 WHERE hotel_id="{hotel_id}"'
        )
        db.commit()

        #store to local file
        hotel_detail_path = os.path.join(hotel_id_dir, f'{hotel_id}_hotel_detail.json') # hotel_detail local file path
        with open(hotel_detail_path, 'w') as json_file:
            json.dump(hotel_detail, json_file, indent=7)
    time.sleep(5)
    
    
def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()



# COLLECT HOTEL REVIEW

In [21]:
from tripadvisor import Tripadvisor
import pymongo
from pymongo import MongoClient
import mysql.connector
import time
import config
import os
import json
import random


def get_all_review(hotel_id):
    tripadvisor = Tripadvisor()
    
    offset = 0
    all_hotel_review = list()
    while True:
        hotel_review_raw = tripadvisor.hotel_review(hotel_id, offset)
        hotel_review_data = [data for data in hotel_review_raw['data'] if 'ad_context' not in data]
        total_result = int(hotel_review_raw['paging']['total_results'])
        if len(hotel_review_data) != 50:
            diff = total_result - len(all_hotel_review)
            if diff > 50:
                raise Exception('REVIEW COUNT NOT 50, SOMETHING WRONG')
        all_hotel_review += hotel_review_data
        offset += 50
        print(f'\rCurrent review count: {len(all_hotel_review):<6} | Total results: {total_result:<6}', end='')
        if len(all_hotel_review) == total_result:
            break
        time.sleep(random.uniform(5.0, 10.0))
        continue
    return all_hotel_review


def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()


mongo_cluster = MongoClient('mongodb://localhost:27017')
mongo_db = mongo_cluster['tripadvisor']
mongo_collection = mongo_db['hotel_review']

hotel_data_dir = './hotel_data'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

# select all hotel list that have not been parsed
hotel_list = db_select(db, "SELECT * FROM hotel_list WHERE hotel_review_parsed=0 LIMIT 44")


# iterate per hotel to get hotel review then change hotel_review_parsed to True
for hotel in hotel_list:
    hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed = hotel

    # create hotel dir if not exist
    hotel_id_dir = os.path.join(hotel_data_dir, hotel_id)
    if not os.path.exists(hotel_id_dir):
        os.makedirs(hotel_id_dir)

    # get hotel review from tripadvisor api
    hotel_reviews = get_all_review(hotel_id)
    
    # insert data into mongodb
    # mongo_post = dict()
    # mongo_post['_id'] = hotel_id
    # mongo_post['data'] = hotel_reviews
    # mongo_collection.insert_one(mongo_post)

    #store and update database
    cursor.execute(
        'INSERT IGNORE INTO hotel_review (hotel_id, hotel_reviews) VALUES (%s, %s)',
        (hotel_id, json.dumps(hotel_reviews))
    )
    cursor.execute(
        f'UPDATE hotel_list SET hotel_review_parsed=1 WHERE hotel_id="{hotel_id}"'
    )
    db.commit()

    #store to local file
    hotel_review_path = os.path.join(hotel_id_dir, f'{hotel_id}_hotel_review.json') # hotel_review local file path
    with open(hotel_review_path, 'w') as json_file:
        json.dump(hotel_reviews, json_file, indent=7)
    time.sleep(5)
    




Current review count: 1500   | Total results: 1807  

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

# ALTER JSON FILE AND UPLOAD TO MYSQL SERVER (ARCHIVED)

In [ ]:
import mysql.connector
import config
import json



destination_path = './hotel_data/hotel_list.json'

db = init_db()
cursor = get_cursor(db)

temp_list = list()
with open('hotel_list.json') as json_file:
    json_data = json.load(json_file)
    
    for hotel in json_data:
        cursor.execute(
            "INSERT IGNORE INTO hotel_list (hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed) VALUES (%s, %s, %s, %s);",
            (hotel['hotel_id'], hotel['link'], False, False )
            )
        db.commit()

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()



# MIGRATE DATA FROM LOCAL TO MONGODB (ARCHIVED)


In [12]:
import pymongo
from pymongo import MongoClient
import json
import os


hotel_data_dir = './hotel_data'
hotel_detail_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_detail.json') for dirr in os.listdir(hotel_data_dir)]

cluster = MongoClient('mongodb://localhost:27017')
db = cluster['tripadvisor']
collection = db['hotel_detail']

for detail in hotel_detail_paths:
    with open(detail) as json_file:
        json_data = json.load(json_file)
        new_data = dict()
        new_data['_id'] = json_data['data'][0]['location_id']
        new_data['data'] = json_data['data'][0]
        collection.insert_one(new_data)
       






In [21]:
import pymongo
from pymongo import MongoClient
import json
import os


hotel_data_dir = './hotel_data'
hotel_review_paths = [os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_review.json') for dirr in os.listdir(hotel_data_dir) if os.path.exists(os.path.join(hotel_data_dir,dirr,f'{dirr}_hotel_review.json'))]

cluster = MongoClient('mongodb://localhost:27017')
db = cluster['tripadvisor']
collection = db['hotel_review']

for hotel_review_path in hotel_detail_paths:
    with open(hotel_review_path) as json_file:
        review_data = json.load(json_file)
        for i,review in enumerate(review_data):
            new_data = dict()
            new_data['_id'] = review['id']
            new_data['hotel_id'] = review['location_id']
            new_data['data'] = review
            collection.insert_one(new_data)
# for detail in hotel_detail_paths:
#     with open(detail) as json_file:
#         json_data = json.load(json_file)
#         new_data = dict()
#         new_data['_id'] = json_data['data'][0]['location_id']
#         new_data['data'] = json_data['data']
#         collection.insert_one(new_data)